# Challenge Data Engineer LATAM - Juan Felipe Hurtado

- El archivo json se tiene en el directorio raiz de este archivo.

# Importaciones globales del reto:

In [1]:
import os
import pandas as pd
import ujson as json
import emoji
import re
from collections import Counter
from typing import List, Tuple
from datetime import datetime
from collections import defaultdict

# 1 Las top 10 fechas donde hay más tweets. 
Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.
- Enfoque: Tiempo de ejecucion optimizado.

- Descripción:
Este código define una función llamada q1_time que procesa un archivo. Si existe un archivo Parquet más reciente que el archivo original, lo lee; de lo contrario, lee el archivo JSON, realiza manipulaciones de datos y lo guarda como Parquet. Luego, identifica las 10 fechas con más apariciones en el archivo original y, para cada una de ellas, encuentra el nombre de usuario con más apariciones en esa fecha. El resultado final es una lista de tuplas que contienen la fecha y el nombre de usuario correspondiente para las 10 fechas principales.

In [2]:


def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Divide el file_path en nombre de archivo y extensión
    file_name = os.path.splitext(file_path)
    
    # Crea el nombre del archivo Parquet
    parquet_file = file_name + '.parquet'
    
    # Verifica si existe el archivo Parquet y si es más reciente que el archivo original
    if os.path.exists(parquet_file) and os.path.getmtime(parquet_file) >= os.path.getmtime(file_path):
        # Lee el archivo Parquet si cumple la condición
        df = pd.read_parquet(parquet_file)
    else:
        # Lee el archivo JSON y realiza manipulaciones de datos si no cumple la condición
        df = pd.read_json(file_path, lines=True)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['username'] = df['user'].apply(lambda x: x.get('username'))
        del df['user']
        df.to_parquet(parquet_file, index=False)

    # Agrupa por fecha y nombre de usuario, cuenta y obtiene la fila con el máximo conteo por fecha
    top_users_df = df.groupby(['date', 'username']).size().reset_index(name='count')
    top_users_df = top_users_df.loc[top_users_df.groupby('date')['count'].idxmax()]
    
    # Encuentra las 10 fechas con más apariciones y su recuento
    top_dates_df = df['date'].value_counts().nlargest(10).reset_index()
    
    # Crea una lista de tuplas con la fecha y el nombre de usuario correspondiente para las 10 fechas principales
    result = [(row['index'], top_users_df[top_users_df['date'] == row['index']]['username'].values[0]) for _, row in top_dates_df.iterrows()]
    
    return result

 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q1_memory que procesa un archivo de texto. Lee cada línea del archivo, interpreta cada línea como un objeto JSON que representa un tweet, y realiza un seguimiento de la fecha y el nombre de usuario más frecuentes para cada fecha. Luego, encuentra las 10 fechas con mayor número de usuarios únicos y devuelve una lista de tuplas que contienen la fecha y el nombre de usuario más frecuente para cada una de esas fechas.

In [3]:


def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Crear un diccionario anidado para contar usuarios por fecha
    usuarios_por_fecha = defaultdict(lambda: defaultdict(int))
    # Abrir el archivo en modo lectura
    with open(file_path, "r") as archivo:
        # Iterar sobre cada línea en el archivo
        for linea in archivo:
            # Cargar cada línea como un objeto JSON
            tweet = json.loads(linea)
            # Extraer la fecha del tweet y convertirla en un objeto de fecha
            fecha_tweet_str = tweet['date'].split("T")[0]
            fecha_tweet = datetime.strptime(fecha_tweet_str, '%Y-%m-%d').date()
            # Obtener el nombre de usuario del tweet
            nombre_usuario = tweet['user']['username']
            # Actualizar el contador de usuarios por fecha
            usuarios_por_fecha[fecha_tweet][nombre_usuario] += 1
    # Inicializar una lista vacía para almacenar las fechas y usuarios principales
    fechas_usuarios_principales = []
    # Iterar a través de las fechas y sus recuentos de usuarios
    for fecha, recuentos_usuarios in usuarios_por_fecha.items():
        # Encontrar el usuario con el recuento máximo para cada fecha
        usuario_principal = max(recuentos_usuarios, key=recuentos_usuarios.get)
        # Agregar la fecha y el usuario principal a la lista
        fechas_usuarios_principales.append((fecha, usuario_principal))
    # Ordenar la lista por recuento de usuarios en orden descendente y tomar las 10 primeras entradas
    resultado = sorted(fechas_usuarios_principales, key=lambda x: usuarios_por_fecha[x[0]][x[1]], reverse=True)[:10]
    return resultado


# 2. Los top 10 emojis más usados con su respectivo conteo.
- Enfoque: Tiempo de ejecucion optimizado
- Descripción: Este código define una función llamada q2_time que procesa un archivo. Primero, verifica si existe un archivo Parquet con el mismo nombre que el archivo JSON de entrada. Si existe, lo lee; de lo contrario, lee el archivo JSON, extrae los emojis de los contenidos de los tweets, cuenta la frecuencia de cada emoji y devuelve una lista de los 10 emojis más comunes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el emoji y su frecuencia de aparición en los tweets del archivo.

In [4]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    # Crea el nombre del archivo Parquet reemplazando la extensión .json por .parquet
    archivo_parquet = file_path.replace('.json', '.parquet')
    
    # Verifica si existe el archivo Parquet
    if os.path.exists(archivo_parquet):
        # Lee el archivo Parquet si existe
        df = pd.read_parquet(archivo_parquet)
    else:
        # Lee el archivo JSON si el Parquet no existe y lo guarda como Parquet
        df = pd.read_json(file_path, lines=True)
        df.to_parquet(archivo_parquet, index=False)
    
    # Inicializa una lista vacía para almacenar todos los emojis en el contenido de los tweets
    todos_emojis = []
    
    # Itera a través de la columna 'content' del DataFrame
    for contenido in df['content']:
        # Extrae los emojis de cada contenido y los agrega a la lista de todos los emojis
        emojis_en_contenido = [entrada['emoji'] for entrada in emoji.emoji_list(contenido)]
        todos_emojis.extend(emojis_en_contenido)
    
    # Cuenta la frecuencia de cada emoji
    conteo_emojis = Counter(todos_emojis)
    
    # Encuentra los 10 emojis más comunes
    top_emojis = conteo_emojis.most_common(10)
    
    return top_emojis


 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q2_memory que procesa un archivo de texto. Lee cada línea del archivo, interpreta cada línea como un objeto JSON que representa un tweet y cuenta la frecuencia de cada emoji en el contenido de los tweets. Luego, devuelve una lista de los 10 emojis más comunes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el emoji y su frecuencia de aparición en los tweets del archivo.

In [5]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    # Inicializa un contador para contar emojis
    conteo_emojis = Counter()
    
    # Abre el archivo en modo lectura
    with open(file_path, 'r') as archivo:
        # Itera sobre cada línea en el archivo
        for linea in archivo:
            # Carga cada línea como un objeto JSON
            tweet = json.loads(linea)
            
            # Obtiene el contenido del tweet o una cadena vacía si no hay contenido
            contenido = tweet.get('content', '')
            
            # Extrae los emojis del contenido del tweet y actualiza el contador de emojis
            emojis_en_contenido = [entrada['emoji'] for entrada in emoji.emoji_list(contenido)]
            conteo_emojis.update(emojis_en_contenido)
    
    # Encuentra los 10 emojis más comunes
    top_emojis = conteo_emojis.most_common(10)
    
    return top_emojis


# 3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. 
- Enfoque: Tiempo de ejecucion optimizado
- Descripción: Este código define una función llamada q3_time que procesa un archivo. Primero, verifica si existe un archivo Parquet con el mismo nombre que el archivo JSON de entrada. Si existe, lo lee; de lo contrario, lee el archivo JSON, encuentra menciones en el contenido de los tweets, cuenta la frecuencia de cada mención y devuelve una lista de las 10 menciones más frecuentes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el nombre de usuario mencionado y la frecuencia de aparición en los tweets del archivo.

In [6]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # Crea el nombre del archivo Parquet reemplazando la extensión .json por .parquet
    archivo_parquet = file_path.replace('.json', '.parquet')
    
    # Verifica si existe el archivo Parquet
    if os.path.exists(archivo_parquet):
        # Lee el archivo Parquet si existe
        df = pd.read_parquet(archivo_parquet)
    else:
        # Lee el archivo JSON si el Parquet no existe y lo guarda como Parquet
        df = pd.read_json(file_path, lines=True)
        df.to_parquet(archivo_parquet, index=False)
    
    # Encuentra menciones en el contenido de los tweets y crea una nueva columna 'mentions'
    df['menciones'] = df['content'].str.findall(r'@(\w+)')
    
    # Divide las filas del DataFrame por cada mención, creando múltiples filas duplicadas
    df = df.explode('menciones', ignore_index=True)
    
    # Calcula la frecuencia de menciones y crea un nuevo DataFrame con columnas 'nombre_de_usuario' y 'conteo'
    conteo_menciones = df['menciones'].value_counts().reset_index()
    conteo_menciones.columns = ['nombre_de_usuario', 'conteo']
    
    # Obtiene las 10 menciones más frecuentes como un iterable de tuplas sin índice
    top_menciones = conteo_menciones.head(10).itertuples(index=False, name=None)
    
    # Convierte el iterable en una lista de tuplas
    return list(top_menciones)


 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q3_memory que procesa un archivo de texto. Lee cada línea del archivo, intenta cargarla como un objeto JSON (capturando excepciones JSONDecodeError para líneas inválidas), busca menciones en el contenido de los tweets y cuenta la frecuencia de cada mención. Luego, devuelve una lista de las 10 menciones más frecuentes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el nombre de usuario mencionado y la frecuencia de aparición en los tweets del archivo.

In [7]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    # Inicializa un contador para contar menciones
    conteo_menciones = Counter()
    
    # Abre el archivo en modo lectura
    with open(file_path, 'r') as archivo:
        # Itera sobre cada línea en el archivo
        for linea in archivo:
            try:
                # Intenta cargar cada línea como un objeto JSON (puede haber excepciones JSONDecodeError)
                tweet = json.loads(linea)
                
                # Obtiene el contenido del tweet o una cadena vacía si no hay contenido
                contenido = tweet.get('content', '')
                
                # Encuentra menciones en el contenido del tweet
                menciones = re.findall(r'@(\w+)', contenido)
                
                # Actualiza el contador de menciones con las menciones encontradas
                conteo_menciones.update(menciones)
            except json.JSONDecodeError:
                # Captura excepciones JSONDecodeError en caso de líneas inválidas
                pass
    
    # Obtiene las 10 menciones más frecuentes
    top_menciones = conteo_menciones.most_common(10)
    
    return top_menciones


# Resultados


# Question 1

# Question 2

# Question 3